In [4]:
from agential.cog.agent.reflexion import ReflexionCoTAgent
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_COT_FEWSHOT_EXAMPLES,
    REFLEXION_COT_INSTRUCTION,
    REFLEXION_COT_REFLECT_FEWSHOT_EXAMPLES,
    REFLEXION_COT_REFLECT_INSTRUCTION
)

from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings('ignore')

llm = ChatOpenAI()

agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"}
)

In [9]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"
context = 'VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. It was founded for broadcast of VIVA Germany as VIVA Media AG in 1993 and has been owned by their original concurrent Viacom, the parent company of MTV, since 2004. Viva channels exist in some European countries; the first spin-offs were launched in Poland and Switzerland in 2000.\n\nA Gesellschaft mit beschränkter Haftung (] , abbreviated GmbH ] and also GesmbH in Austria) is a type of legal entity very common in Germany, Austria, Switzerland (where it is equivalent to a S.à r.l.) and Liechtenstein. In the United States, the equivalent type of entity is the limited liability company (LLC). The name of the GmbH form emphasizes the fact that the owners ("Gesellschafter", also known as members) of the entity are not personally liable for the company\'s debts. "GmbH"s are considered legal persons under German and Austrian law. Other variations include mbH (used when the term "Gesellschaft" is part of the company name itself), and gGmbH ("gemeinnützige" GmbH) for non-profit companies.'

out = agent.generate(
    question=question, 
    key=key, 
    examples=REFLEXION_COT_FEWSHOT_EXAMPLES,
    reflection_strategy="last_attempt",
    prompt=REFLEXION_COT_INSTRUCTION,
    reflect_examples=REFLEXION_COT_REFLECT_FEWSHOT_EXAMPLES,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION,
    additional_keys={},
    reflection_additional_keys={},
    max_trials=3,
    patience=3,
    reset=True

)

TypeError: ReflexionCoTReflector.reflect() got an unexpected keyword argument 'additional_keys'

In [ ]:
out

[{'thought': 'Let\'s think step by step. VIVA Media AG changed its name to VIVACOM AG in 2004. The new acronym VIVACOM stands for "Vivendi Communications."',
  'action_type': 'Finish',
  'query': 'Vivendi Communications',
  'obs': 'Answer is INCORRECT',
  'answer': 'Vivendi Communications',
  'is_correct': False}]